In [2]:
import os

In [3]:
pwd

'c:\\Users\\Viral Sherathiya\\Desktop\\NLP Data\\Text_Summarizer\\research'

In [4]:
os.chdir('../')

In [5]:
pwd

'c:\\Users\\Viral Sherathiya\\Desktop\\NLP Data\\Text_Summarizer'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir :Path
    source_url :str
    local_data_file :Path
    unzip_dir :Path

In [8]:
from textSummarization.constants import *
from textSummarization.utils.common import  read_yaml,create_directories
class ConfigurationManager:
    
    def __init__(
        self,
        config_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) ->DataIngestionConfig:
        config =self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config
        


In [9]:
import os
import urllib.request as request
import zipfile
from textSummarization import logger
from textSummarization.utils.common import get_size 


In [12]:
import requests
import os
import urllib.request as request
import zipfile
from textSummarization import logger
from textSummarization.utils.common import get_size 

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data_files(self):
        if not os.path.exists(self.config.local_data_file):
            response = requests.get(self.config.source_url)
            if response.status_code == 200:
                with open(self.config.local_data_file, 'wb') as file:
                    file.write(response.content)
                logger.info(f"Downloaded data to {self.config.local_data_file} with size of {get_size(Path(self.config.local_data_file))}")
            else:
                logger.error(f"Failed to download data. HTTP status code: {response.status_code}")
        else:
            logger.info(f"Data file already exists at {self.config.local_data_file} with size of {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_dir = self.config.unzip_dir
        os.makedirs(unzip_dir, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data_files()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-08-13 11:14:53,797] || common.py || 33 || TextSummarizerLogger || read_yaml() || 33 || INFO || yaml file : config\config.yaml loaded successfully
[2023-08-13 11:14:53,805] || common.py || 33 || TextSummarizerLogger || read_yaml() || 33 || INFO || yaml file : config\params.yaml loaded successfully
[2023-08-13 11:14:53,805] || common.py || 68 || TextSummarizerLogger || create_directories() || 68 || INFO || Directory artifacts already exists
[2023-08-13 11:14:53,805] || common.py || 68 || TextSummarizerLogger || create_directories() || 68 || INFO || Directory artifacts/data_ingestion already exists
[2023-08-13 11:14:53,805] || 3695360171.py || 16 || TextSummarizerLogger || download_data_files() || 16 || INFO || Data file already exists at artifacts/data_ingestion/data.zip with size of ~ 7718 KB
